In [1]:
pip install beautifulsoup4 requests pandas

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_venue_details(match_url):
    """
    Scrape cricket match venue details from a given URL
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(match_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    venue_data = {
        'match_venue_stadium': None,
        'match_venue_city': None,
        'match_venue_capacity': None,
        'match_venue_host_teams': None
    }
    
    try:
        venue_info = soup.find('div', class_='match-info')
        if venue_info:
            venue_parts = venue_info.text.strip().split(',')
            if len(venue_parts) >= 2:
                venue_data['match_venue_stadium'] = venue_parts[0].strip()
                venue_data['match_venue_city'] = venue_parts[1].strip()
        capacity_tag = soup.find('span', string=lambda x: x and 'capacity' in x.lower())
        if capacity_tag:
            venue_data['match_venue_capacity'] = capacity_tag.find_next('span').text.strip()
        teams_tag = soup.find('span', string=lambda x: x and 'home team' in x.lower())
        if teams_tag:
            venue_data['match_venue_host_teams'] = teams_tag.find_next('span').text.strip()
        ground_link = soup.find('a', href=lambda x: x and 'ground' in x)
        if ground_link:
            ground_url = 'https://www.espncricinfo.com' + ground_link['href']
            ground_response = requests.get(ground_url, headers=headers)
            ground_soup = BeautifulSoup(ground_response.content, 'html.parser')
            capacity_label = ground_soup.find('span', string='Capacity')
            if capacity_label:
                venue_data['match_venue_capacity'] = capacity_label.find_next('span').text.strip()
            teams_label = ground_soup.find('span', string='Home team')
            if teams_label:
                venue_data['match_venue_host_teams'] = teams_label.find_next('span').text.strip()   
    except Exception as e:
        print(f"Error scraping venue data: {e}")   
    return venue_data
def scrape_match_with_venue(match_url):
    """
    Only scrape venue details.
    """
    venue_data = scrape_venue_details(match_url)
    return venue_data
if __name__ == "__main__":
    match_url = "https://www.espncricinfo.com/series/ipl-2023-1345038/chennai-super-kings-vs-gujarat-titans-final-1370353/full-scorecard"
    match_data = scrape_match_with_venue(r"https://www.espncricinfo.com/series/ipl-2023-1345038/chennai-super-kings-vs-gujarat-titans-final-1370353/full-scorecard")
    df = pd.DataFrame([match_data])
    print(df[['match_venue_stadium', 'match_venue_city', 'match_venue_capacity', 'match_venue_host_teams']])
def scrape_scorecard(match_url):
    """
    Dummy implementation. Replace with actual scorecard scraping logic.
    """
    return {
        'team1': 'Chennai Super Kings',
        'team2': 'Gujarat Titans',
        'score_team1': '171/5',
        'score_team2': '214/4',
        'result': 'Chennai Super Kings won by 5 wickets (DLS method)'
    }


  match_venue_stadium match_venue_city match_venue_capacity  \
0                None             None                 None   

  match_venue_host_teams  
0                   None  
